Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [11]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from nptyping import NDArray, Shape, Int, Bool
from scipy import sparse

In [12]:
seed_initializer = 2654435761

def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points * seed_initializer + num_sets + density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [13]:
x = make_set_covering_problem(1000, 1000, .3)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: False


In [14]:
covering_sets: sparse.lil_array = make_set_covering_problem(num_points=5, num_sets=10, density=.1)

In [15]:
print(covering_sets)

  (0, 3)	True
  (2, 2)	True
  (2, 4)	True
  (3, 1)	True
  (3, 2)	True
  (4, 1)	True
  (4, 2)	True
  (4, 4)	True
  (5, 1)	True
  (5, 4)	True
  (7, 1)	True
  (8, 0)	True
  (9, 0)	True
  (9, 1)	True


In [16]:
def overlap_sets(covering_sets: sparse.lil_array) -> NDArray[Shape["1, ..."], Bool]:
    sum: NDArray[Shape["1, ..."], Int] = covering_sets.sum(axis=0)
    boolean_array_mapper = np.vectorize(lambda x: x > 0)
    return boolean_array_mapper(sum)

In [17]:
""" def filter_covering_sets(covering_sets: sparse.lil_array, rows_indices: list[int]) -> sparse.lil_array:
    matrix = sparse.lil_matrix(covering_sets)
    filtered_matrix = matrix[rows_indices, :]
    return sparse.lil_array(filtered_matrix) """

In [18]:
def fitness(state: list[bool]):
    taken_sets_indices: list[int] = [i for i, x in enumerate(state) if x]
    # taken_sets: sparse.lil_array = filter_covering_sets(covering_sets, taken_sets_indices)
    taken_sets: sparse.lil_array = covering_sets[taken_sets_indices, :] # type: ignore
    reduced = overlap_sets(taken_sets)
    coverage = sum(reduced)
    cost = len(taken_sets_indices)
    return (coverage, -cost)

In [19]:
print(fitness([True, False, False, True, False, False, False, False, False, False]))

(3, -2)
